# Section 2 Risk Factor Extraction

In section 1, we have downloaded the fincancial reports from 2019Q1 to 2023Q3. In this section, we are going to extract the risk factor text from each of these files. According to the requirement of the 10-K form given by SEC (https://www.sec.gov/files/form10-k.pdf), there is an section named "Item 1A. Risk Factors" included in the form  which shows the potential risks faced by the company. Therefore, it is important for us to filter and cleanse the text from this part so that we can use it for further analysis.

In [1]:
import pandas as pd
import numpy as np
import requests
import os
import re
import html
import warnings
warnings.filterwarnings("ignore")

# 2.3 Technology Service Industry (7370, 7372, 7374)

In this section, we only focus on companies in Technology Service Industry.

## 2.3.1 Industry 7370

### 1. Read in the files and store in a dictionary

In [2]:
folderPath = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Downloaded Financial Reports/Technology Service Industry/Industry_7370/"
fileName_lst = os.listdir(folderPath)
filePair_dict = {} # filePair stores the file name and the corresponding file text.
for fileName in fileName_lst:  
    # encode each document
    filePath = str(folderPath) + str(fileName)    
    with open(filePath, 'r', encoding='utf-8') as file:
        fileText = file.read()
    filePair_dict[fileName] = fileText

In [3]:
filePair_dict.keys()

dict_keys(['0000746210-23-000011.txt', '0000891103-23-000013.txt', '0000950170-23-003200.txt', '0000950170-23-003212.txt', '0000950170-23-004649.txt', '0000950170-23-005080.txt', '0000950170-23-005868.txt', '0000950170-23-008474.txt', '0001096906-22-000930.txt', '0001096906-23-000237.txt', '0001140361-23-011303.txt', '0001171843-22-002601.txt', '0001213900-22-017545.txt', '0001213900-22-019613.txt', '0001213900-22-020136.txt', '0001213900-23-023973.txt', '0001213900-23-024638.txt', '0001326801-23-000013.txt', '0001327318-23-000006.txt', '0001410578-22-001567.txt', '0001422930-23-000008.txt', '0001437749-23-004908.txt', '0001437749-23-005106.txt', '0001454938-23-000029.txt', '0001475115-23-000040.txt', '0001493152-22-008815.txt', '0001493152-22-009607.txt', '0001493152-22-010132.txt', '0001493152-23-010178.txt', '0001506293-23-000023.txt', '0001553350-22-000730.txt', '0001558370-23-002596.txt', '0001558891-23-000023.txt', '0001564408-23-000013.txt', '0001575872-23-000447.txt', '00015829

### 2. Extract the raw risk factor text

In [4]:
def remove_tags(text):
    # replace the tag included by <> to a \ 
    pattern1 = r'<[^>]*>'
    cleanText = re.sub(pattern1, "|", text)
    
    # replace the multiple continued \ to a single \
    cleanText = re.sub(r'\s*\|+\s*', '|', cleanText)
    cleanText = re.sub(r'\|+', '|', cleanText)
    
    # replace the unicode
    cleanText = html.unescape(cleanText)
    
    # remove the line break
    cleanText = cleanText.replace('\n', '')
    # remove the pure numbers text between \
    cleanText = re.sub(r'\|(\d+)\|', '|', cleanText)
    # remove the unrecognized string
    cleanText = re.sub(r'|\xa0\|', '', cleanText)
    cleanText = re.sub(r'|\xa0|', '', cleanText)
    return cleanText

In [5]:
def separate_main_body(text):
    lastContentInTC_lst = [">form 10-k summary<", ">item 16. form 10-k summary<", r">exhibits, financial statement schedules<", 
                           r">exhibits and financial statement schedules<", r"exhibits and financial statement schedules <",
                           ">exhibits<"]
    start_idx = -1
    for content in lastContentInTC_lst:
        if content in text.lower():
            start_idx = text.lower().find(content)
            break
            
    if start_idx == -1:
        return False
    
    mainBody = text[start_idx:]
    mainBody = html.unescape(mainBody)
    return mainBody

In [6]:
def extract_risk_factors(mainBody, fileName):
    # match the risk factor
    keywordMatch = re.findall(r">item 1a.\s+risk factors\.?\<", mainBody.lower())
    
    # if fail to match the pattern, the pattern might be segmented be some html tags, so re-match
    if not keywordMatch:
        keyword = seperated_keyword_rematch(mainBody)
        
    # if match the pattern successfully, just keep the keyword of risk factor
    else:
        keyword = keywordMatch[0]
    
    # check if match successfully
    if not keyword:
        return False
    keywordCheck_lst = ["item", r"item 1a\.?", "risk", "factors", "risk factors"]
    if keyword in keywordCheck_lst:
        print("Warning: It is better to check whether the extraction is correct manually for file", {fileName})
    
    # match the next section of the risk factor
    nextSectionPattern_lst = [r">item 1b.\s+unresolved staff comments\.?\<", r">item 2.\s+description of property\.?\<"]
    for pattern in nextSectionPattern_lst:
        nextSectionMatch = re.findall(pattern, mainBody.lower())
        
        # if fail to match the pattern, the pattern might be segmented be some html tags, so re-match
        if not nextSectionMatch:
            nextSection = seperated_next_section_rematch(mainBody, pattern)
            # if re-match successfully, jump out of the loop
            if nextSection:
                break
        
        # if match the pattern successfully, just keep the next section keyword and jump out of the loop
        else:
            nextSection = nextSectionMatch[0]
            break
    
    # check if match successfully
    if not nextSection:
        return False
    
    # if match successfully, extract the risk factor
    start_idx = mainBody.lower().find(keyword) # start index is the position of the keyword occured in the main body
    end_idx = mainBody.lower().find(nextSection) # end index is the position of the next section begin in the main body
    riskFactors = mainBody[start_idx: end_idx]
    return riskFactors

In [7]:
def seperated_keyword_rematch(mainBody):
    cleanedMainBody = remove_tags(mainBody)
    keywordMatch = re.findall(r"i\|?t\|?e\|?m\|?\s+\|?1\|?a\|?.\|?\s+\|?r\|?i\|?s\|?k\|?\s+\|?f\|?a\|?c\|?t\|?o\|?r\|?s\.?", cleanedMainBody.lower())
    
    # check if re-match the keyword of risk factor successfully
    if not keywordMatch:
        return False
    
    # if re-match successfully, use the longer segmentation as the keyword of the risk factor
    reMatchedKeyword = max(keywordMatch[0].split('|'), key=len)
    return reMatchedKeyword
    
def seperated_next_section_rematch(mainBody, nextSectionPattern):
    # cleanse the main body text
    cleanedMainBody = remove_tags(mainBody)
    
    # edit the pattern 
    edittedPattern = nextSectionPattern.replace(r">", "").replace(r"\s+", " ").replace(r"\.?\<", "")
    edittedPattern = "\|?".join(edittedPattern)
    edittedPattern = r"{}\.?".format(edittedPattern)
    pattern = edittedPattern.replace(" ", "\s?+")
    nextSectionMatch = re.findall(pattern, cleanedMainBody.lower())
    
    # check if re-match the keyword of next section successfully
    if not nextSectionMatch:
        return False
    
    # if re-match successfully, use the longer segmentation as the keyword of the next section 
    reMatchedNextSection = max(nextSectionMatch[0].split('|'), key=len)
    return reMatchedNextSection

In [8]:
for key, value in filePair_dict.items():
    fileName = key
    fileText = value
    mainBody = separate_main_body(fileText) # separate the original text as the table of content and the main body.
    
    if not mainBody:
        print("The extraction is failed for", {fileName}, "because of the error in seperating main body.")
        continue
    
    riskText = extract_risk_factors(mainBody, fileName) # extract the risk related text
    if not riskText:
        print("The extraction is failed for", {fileName}, "because of the error in extracting risk factors.")
    else:
        riskFilePath = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Risk Factor Text/Technology Service Industry/Industry_7370/Raw Text/" + fileName
        with open(riskFilePath, 'w', encoding='utf-8') as file:
            file.write(riskText)

The extraction is failed for {'0001140361-23-011303.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001213900-22-017545.txt'} because of the error in seperating main body.
The extraction is failed for {'0001213900-22-019613.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001213900-23-023973.txt'} because of the error in seperating main body.
The extraction is failed for {'0001213900-23-024638.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001326801-23-000013.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001327318-23-000006.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001410578-22-001567.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001493152-22-008815.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001493152-2

### 3. Extract the subjects from raw risk factors text

In [10]:
from bs4 import BeautifulSoup

def extract_subjects(riskFactors, subjectFormat):
    soup = BeautifulSoup(riskFactors, 'html.parser')
    subject_lst = []
    for span in soup.find_all('span'):
        style = span.get('style')
        if style and subjectFormat in style:
            subject_lst.append(span.get_text())
    return subject_lst

In [11]:
rawTextFolderPath = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Risk Factor Text/Technology Service Industry/Industry_7370/Raw Text/"
rawTextFileName_lst = os.listdir(rawTextFolderPath)
rawTextFilePair_dict = {} # filePair stores the file name and the corresponding file text.
for fileName in rawTextFileName_lst:  
    # encode each document
    filePath = str(rawTextFolderPath) + str(fileName)    
    with open(filePath, 'r', encoding='utf-8') as file:
        rawText = file.read()
    rawTextFilePair_dict[fileName] = rawText

In [12]:
subjectFormat_lst = ["font-weight:700", "font-weight:bold"]
for key, value in rawTextFilePair_dict.items():
    fileName = key
    rawText = value
    
    i = 0
    subject_lst = []
    
    while not subject_lst:
        subjectFormat = subjectFormat_lst[i]
        subject_lst = extract_subjects(rawText, subjectFormat)
        i += 1
        if i >= len(subjectFormat_lst):
            break
        
    if not subject_lst:
        print("The subject fail to extract for", {fileName})
    else:
        subjectFolder = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Risk Factor Text/Technology Service Industry/Industry_7370/Subject/"
        subjectFilePath = subjectFolder + str(fileName)
        with open(subjectFilePath, 'w', encoding='utf-8') as file:
            for item in subject_lst:
                file.write(item + '\n')

The subject fail to extract for {'0001096906-22-000930.txt'}
The subject fail to extract for {'0001096906-23-000237.txt'}
The subject fail to extract for {'0001171843-22-002601.txt'}
The subject fail to extract for {'0001213900-22-020136.txt'}
The subject fail to extract for {'0001437749-23-004908.txt'}
The subject fail to extract for {'0001437749-23-005106.txt'}
The subject fail to extract for {'0001493152-22-010132.txt'}
The subject fail to extract for {'0001553350-22-000730.txt'}
The subject fail to extract for {'0001558891-23-000023.txt'}
The subject fail to extract for {'0001640334-23-000500.txt'}
The subject fail to extract for {'0001758699-23-000004.txt'}
The subject fail to extract for {'0001765048-23-000003.txt'}


---

## 2.3.2 Industry 7372

### 1. Read in the files and store in a dictionary

In [13]:
folderPath = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Downloaded Financial Reports/Technology Service Industry/Industry_7372/"
fileName_lst = os.listdir(folderPath)
filePair_dict = {} # filePair stores the file name and the corresponding file text.
for fileName in fileName_lst:  
    # encode each document
    filePath = str(folderPath) + str(fileName)    
    with open(filePath, 'r', encoding='utf-8') as file:
        fileText = file.read()
    filePair_dict[fileName] = fileText

### 2. Extract the raw risk factor text

In [14]:
for key, value in filePair_dict.items():
    fileName = key
    fileText = value
    mainBody = separate_main_body(fileText) # separate the original text as the table of content and the main body.
    
    if not mainBody:
        print("The extraction is failed for", {fileName}, "because of the error in seperating main body.")
        continue
    
    riskText = extract_risk_factors(mainBody, fileName) # extract the risk related text
    if not riskText:
        print("The extraction is failed for", {fileName}, "because of the error in extracting risk factors.")
    else:
        riskFilePath = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Risk Factor Text/Technology Service Industry/Industry_7372/Raw Text/" + fileName
        with open(riskFilePath, 'w', encoding='utf-8') as file:
            file.write(riskText)

The extraction is failed for {'0000712515-22-000011.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000896841-23-000019.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000896878-22-000028.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000910638-23-000009.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000935036-23-000009.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000950170-22-018351.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000950170-23-003943.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000950170-23-004143.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0000950170-23-005490.txt'} because of the error in seperating main body.
The extraction is failed for {'000095017

The extraction is failed for {'0001591698-22-000094.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001628280-22-018227.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001628280-23-004842.txt'} because of the error in seperating main body.
The extraction is failed for {'0001640147-22-000023.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001640334-22-000876.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001683168-22-002671.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001683168-22-002694.txt'} because of the error in seperating main body.
The extraction is failed for {'0001683168-22-002719.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001683168-22-002722.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001710495-2

### 3. Extract the subjects from raw risk factors text

In [15]:
rawTextFolderPath = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Risk Factor Text/Technology Service Industry/Industry_7372/Raw Text/"
rawTextFileName_lst = os.listdir(rawTextFolderPath)
rawTextFilePair_dict = {} # filePair stores the file name and the corresponding file text.
for fileName in rawTextFileName_lst:  
    # encode each document
    filePath = str(rawTextFolderPath) + str(fileName)    
    with open(filePath, 'r', encoding='utf-8') as file:
        rawText = file.read()
    rawTextFilePair_dict[fileName] = rawText

In [16]:
subjectFormat_lst = ["font-weight:700", "font-weight:bold"]
for key, value in rawTextFilePair_dict.items():
    fileName = key
    rawText = value
    
    i = 0
    subject_lst = []
    
    while not subject_lst:
        subjectFormat = subjectFormat_lst[i]
        subject_lst = extract_subjects(rawText, subjectFormat)
        i += 1
        if i >= len(subjectFormat_lst):
            break
        
    if not subject_lst:
        print("The subject fail to extract for", {fileName})
    else:
        subjectFolder = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Risk Factor Text/Technology Service Industry/Industry_7372/Subject/"
        subjectFilePath = subjectFolder + str(fileName)
        with open(subjectFilePath, 'w', encoding='utf-8') as file:
            for item in subject_lst:
                file.write(item + '\n')

The subject fail to extract for {'0001017386-22-000150.txt'}
The subject fail to extract for {'0001213900-22-017159.txt'}
The subject fail to extract for {'0001213900-22-020167.txt'}
The subject fail to extract for {'0001213900-22-059573.txt'}
The subject fail to extract for {'0001213900-22-059740.txt'}
The subject fail to extract for {'0001213900-23-024677.txt'}
The subject fail to extract for {'0001213900-23-025215.txt'}
The subject fail to extract for {'0001213900-23-025417.txt'}
The subject fail to extract for {'0001477932-22-002284.txt'}
The subject fail to extract for {'0001493152-22-009403.txt'}
The subject fail to extract for {'0001493152-23-008554.txt'}
The subject fail to extract for {'0001520138-22-000237.txt'}
The subject fail to extract for {'0001553350-23-000241.txt'}
The subject fail to extract for {'0001640334-22-001828.txt'}
The subject fail to extract for {'0001640334-23-000444.txt'}
The subject fail to extract for {'0001683168-22-002295.txt'}
The subject fail to extr

---

## 2.3.3 Industry 7374

### 1. Read in the files and store in a dictionary

In [17]:
folderPath = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Downloaded Financial Reports/Technology Service Industry/Industry_7374/"
fileName_lst = os.listdir(folderPath)
filePair_dict = {} # filePair stores the file name and the corresponding file text.
for fileName in fileName_lst:  
    # encode each document
    filePath = str(folderPath) + str(fileName)    
    with open(filePath, 'r', encoding='utf-8') as file:
        fileText = file.read()
    filePair_dict[fileName] = fileText

### 2. Extract the raw risk factor text

In [18]:
for key, value in filePair_dict.items():
    fileName = key
    fileText = value
    mainBody = separate_main_body(fileText) # separate the original text as the table of content and the main body.
    
    if not mainBody:
        print("The extraction is failed for", {fileName}, "because of the error in seperating main body.")
        continue
    
    riskText = extract_risk_factors(mainBody, fileName) # extract the risk related text
    if not riskText:
        print("The extraction is failed for", {fileName}, "because of the error in extracting risk factors.")
    else:
        riskFilePath = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Risk Factor Text/Technology Service Industry/Industry_7374/Raw Text/" + fileName
        with open(riskFilePath, 'w', encoding='utf-8') as file:
            file.write(riskText)

The extraction is failed for {'0001096906-23-000692.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001140361-23-014888.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001213900-23-021050.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001288847-23-000029.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001327811-22-000030.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001437749-23-004945.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001477932-22-002022.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001477932-23-000012.txt'} because of the error in extracting risk factors.
The extraction is failed for {'0001477932-23-002072.txt'} because of the error in extracting risk factors.
The extraction is failed for {'000149

### 3. Extract the subjects from raw risk factors text

In [19]:
rawTextFolderPath = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Risk Factor Text/Technology Service Industry/Industry_7374/Raw Text/"
rawTextFileName_lst = os.listdir(rawTextFolderPath)
rawTextFilePair_dict = {} # filePair stores the file name and the corresponding file text.
for fileName in rawTextFileName_lst:  
    # encode each document
    filePath = str(rawTextFolderPath) + str(fileName)    
    with open(filePath, 'r', encoding='utf-8') as file:
        rawText = file.read()
    rawTextFilePair_dict[fileName] = rawText

In [20]:
subjectFormat_lst = ["font-weight:700", "font-weight:bold"]
for key, value in rawTextFilePair_dict.items():
    fileName = key
    rawText = value
    
    i = 0
    subject_lst = []
    
    while not subject_lst:
        subjectFormat = subjectFormat_lst[i]
        subject_lst = extract_subjects(rawText, subjectFormat)
        i += 1
        if i >= len(subjectFormat_lst):
            break
        
    if not subject_lst:
        print("The subject fail to extract for", {fileName})
    else:
        subjectFolder = "E:/Jupyter Notebook Files/ISE 540_Text Analytics/Project/Risk Factor Text/Technology Service Industry/Industry_7374/Subject/"
        subjectFilePath = subjectFolder + str(fileName)
        with open(subjectFilePath, 'w', encoding='utf-8') as file:
            for item in subject_lst:
                file.write(item + '\n')

The subject fail to extract for {'0001065949-22-000047.txt'}
The subject fail to extract for {'0001065949-23-000017.txt'}
The subject fail to extract for {'0001096906-22-000873.txt'}
The subject fail to extract for {'0001096906-23-000648.txt'}
The subject fail to extract for {'0001185185-23-000161.txt'}
The subject fail to extract for {'0001213900-23-021526.txt'}
The subject fail to extract for {'0001213900-23-023197.txt'}
The subject fail to extract for {'0001410578-23-000153.txt'}
The subject fail to extract for {'0001437749-23-006745.txt'}
The subject fail to extract for {'0001477932-22-002301.txt'}
The subject fail to extract for {'0001477932-22-002322.txt'}
The subject fail to extract for {'0001477932-22-002481.txt'}
The subject fail to extract for {'0001477932-22-004928.txt'}
The subject fail to extract for {'0001477932-22-007238.txt'}
The subject fail to extract for {'0001493152-23-007879.txt'}
The subject fail to extract for {'0001731122-23-000536.txt'}
The subject fail to extr